<a href="https://colab.research.google.com/github/quinmeh/wildfire_detection/blob/main/wildfire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://githubtocolab.com/gee-community/geemap/blob/master/examples/notebooks/35_geemap_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install geemap

## Import geemap library

The [geemap](https://github.com/gee-community/geemap) Python package has two plotting backends: [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium). A key difference between folium and ipyleaflet is that ipyleaflet is built upon ipywidgets and allows bidirectional communication between the front-end and the backend enabling the use of the map to capture user input, while folium is meant for displaying static data only ([source](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a)). Note that Google Colab currently does not support ipyleaflet ([source](https://github.com/googlecolab/colabtools/issues/60#issuecomment-596225619)). Therefore, if you are using geemap with Google Colab, geemap will automatically use the folium plotting backend.

In [ ]:
import ee
import geemap

## Create an interactive map

In [ ]:
Map = geemap.Map()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=yO9_J957rafV_P6c8_Us2Ip-J92Un3iCAKOQiTbpbqs&tc=f0kAZ8R79p-gsvp1E051aOgv06Kugginx22NjF--FOY&cc=KQuwVpokeGixHvGifvmVDj5zrZL2pDqIlb1psH8uHks

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdLIrYfo-2WhjjRtHhqFmI81S8n3HwqYQRkCAtb6Hg8_EU1TYuwyrNyVzhQ

Successfully saved authorization token.


## Add Earth Engine data

In [ ]:
# dataset = ee.ImageCollection('MODIS/061/MCD43A4') \
#            .filterDate('2020-08-20', '2020-08-25')

# true_color = dataset.select([
#     'Nadir_Reflectance_Band7', 'Nadir_Reflectance_Band6',
#     'Nadir_Reflectance_Band1'
# ])

# true_color_vis = {
#     'min': 0.0,
#     'max': 4000.0,
#     'gamma': 1.4

# }
# Map = geemap.Map()

# Map.setCenter(-122.673, 39.776, 12)
# Map.addLayer(true_color, true_color_vis, 'True Color')

# Map

In [ ]:

# Set the map's center to Davis, CA, with an appropriate zoom level
Map.setCenter(-122.673, 39.776, 12)  # Longitude, Latitude, Zoom Level
# Sentinel-2
sentinel_2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", 10) \
    .filterDate('2020-09-10', '2020-09-25') \
    .median()  # Using median to reduce cloud cover by taking the median value of the stack
sentinel_2_vis = {
    'min': 0.0,
    'max': 4000,
    'bands': ['B12', 'B11', 'B4'],
}
Map.addLayer(sentinel_2, sentinel_2_vis, "Sentinel-2 RGB", True, 1)
#MODIS
# Define the image collection and filter by date
MODIS = ee.ImageCollection('MODIS/061/MCD43A4') \
    .filterDate('2020-09-10', '2020-09-30')
# Select the bands for true color composite
SWIR = MODIS.select([
  'Nadir_Reflectance_Band7', 'Nadir_Reflectance_Band6',
  'Nadir_Reflectance_Band1'
])
# Define visualization parameters
MODIS_vis = {
  'min': 0.0,
  'max': 4000.0,
  'gamma': 1.4,
}
# Add the true color layer to the map
Map.addLayer(SWIR, MODIS_vis, 'MODIS')
# Display the map
Map.addLayerControl()
Map

Map(bottom=3190134.0, center=[39.776, -122.673], controls=(WidgetControl(options=['position', 'transparent_bg'…

In [ ]:
# Draw any shapes on the map using the Drawing tools before executing this code block
feature = Map.draw_last_feature

# Ensure you have a valid feature drawn on the map
if feature is None:
    raise ValueError("Please draw a feature on the map.")

roi = feature.geometry()
roi

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Feature.geometry",
    "arguments": {
      "feature": {
        "functionInvocationValue": {
          "functionName": "Feature",
          "arguments": {
            "geometry": {
              "functionInvocationValue": {
                "functionName": "GeometryConstructors.Polygon",
                "arguments": {
                  "coordinates": {
                    "constantValue": [
                      [
                        [
                          -122.930489,
                          40.049168
                        ],
                        [
                          -122.930489,
                          40.057972
                        ],
                        [
                          -122.91873,
                          40.057972
                        ],
                        [
                          -122.91873,
                          40.049168
                        ],
                        [
                          -122.930489,
                          40.049168
                        ]
                      ]
                    ]
                  },
                  "geodesic": {
                    "constantValue": false
                  }
                }
              }
            }
          }
        }
      }
    }
  }
})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Specify your Google Drive folder path
import os
out_dir = '/content/drive/Shareddrives/ESEARCH_Spring_2024'

# Check if the folder exists, if not, create it
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
def export_image(image_collection, satellite_name, scale):
    filename = os.path.join(out_dir, f"{satellite_name}_Image.tif")

    # Check if the object is an ImageCollection and convert it to an Image if true
    if isinstance(image_collection, ee.imagecollection.ImageCollection):
        image = image_collection.median()  # Using median to combine the images
    else:
        image = image_collection  # It's already an Image, no conversion needed

    # Now apply the clip and unmask operations
    image = image.clip(roi).unmask()

    # Export the image with the specified scale
    geemap.ee_export_image(
        image, filename=filename, scale=scale, region=roi.getInfo(), file_per_band=False
    )

In [ ]:
export_image(sentinel_2, "Sentinel_2-145", 10)  # Sentinel-2 with a scale of 10
export_image(MODIS, "MODIS-145", 500)  # MODIS with a scale of 500

Generating URL ...
Please wait ...
Data downloaded to /content/drive/Shareddrives/ESEARCH_Spring_2024/Sentinel_2-145_Image.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/Shareddrives/ESEARCH_Spring_2024/MODIS-145_Image.tif


## Display the map